# Лабораторная работа №5

## Импорт библеотек

In [59]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.vocab import build_vocab_from_iterator
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


import pandas as pd
import numpy as np

import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords


from tqdm.auto import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Функция Обучения

In [60]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


## Google Drive if need

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Device

In [62]:
device = torch.device('cuda')

## Задание 1. Изучите технологии attention и архитектуры нейронных сетей трансформеров.

## Задание 2. Примените один из трансформеров, например BERT к задаче классификации отзывов клиентов. Сравните полученные результаты с классическими методами машинного обучения, с RNN. Сделайте выводы.

### Dataset

In [63]:
df_reviews = pd.read_csv('/content/drive/MyDrive/NLP_FU/lem_500kdata.csv', index_col = 0)
df_reviews.dropna(inplace = True)
df_reviews

,rating,text
0,3,московский квартал шумно лето ночь дикий гонка...
1,5,замечательный сеть магазин общий хороший ассор...
2,1,знать смутить коготь дать правило удивить хоте...
3,4,хороший условие аренда дружелюбный персонално ...
4,5,топ мастер ангелина топ смысл немного волноват...
...,...,...
499994,5,хороший способ избежать автобусный пытка элект...
499995,4,охрана кривая добрый дверь закрываться автомат...
499996,4,сравнение многий современный платформа эпоха с...
499998,5,семья отличный место рекомендовать это настоящ...


### Balance classes

In [64]:
min_class_size  = df_reviews['rating'].value_counts().min()

df_reviews['rating'].value_counts()

5    389674
4     41114
1     34297
3     21651
2     12071
Name: rating, dtype: int64

In [65]:
df_reviews = pd.concat([group.sample(min_class_size) for _, group in df_reviews.groupby('rating')])

In [66]:
df_reviews

,rating,text
274950,1,плохой доставка который когдалибо пользоваться...
137692,1,здравствуйте советовать посещать данный место ...
84489,1,фармацевт повести некорректно невежливый именн...
86947,1,ужасный отель заселить номер мыший кровать сто...
211105,1,видимо отзыв ниже сильно накрутить андрей иван...
...,...,...
441131,5,реально вкусно ранний мдонпльдс любилатолько к...
420659,5,сервисный центр спутник спасибо огромный решит...
86547,5,еда пиво персонал атмосфера всё шикарно спасибо
421125,5,сервис заправочный станция лукойл высокий уров...


### DeepPavlov tokenizer

In [67]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence", do_lower_case=True)

### Tokenizer

In [68]:
texts = df_reviews['text'].values

tokenized_inputs = []
for text in tqdm(texts, desc="Tokenizing"):
    inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=64, add_special_tokens=True)
    tokenized_inputs.append(inputs)

Tokenizing:   0%|          | 0/60355 [00:00<?, ?it/s]

In [69]:
print(texts[:5])

['плохой доставка который когдалибо пользоваться интервал время доставка вообще непонятно указывать курьер приезжать вздуматься захотеть вечер интервал время доставка приехать ставить факт подъезд утверждать должный звонить минута должный приехать число приезжать третий звонить говорить минута приезжать трубка беретпотом вообще телефон выключить поставить звезда поставить'
 'здравствуйте советовать посещать данный место приобретение цвет первый купить цветок рубль упаковка взять сверху попросить обрезать оба букет обрезать итог упаковка обычный цветной бумага букет замотанный каждый ленточка брать деньга маленький предупредить букет собранный берет рубль упаковка просьба учитывать пожелание упаковывать соответствующе'
 'фармацевт повести некорректно невежливый именно обслуживать спешить течение примерно минута предыдущий клиент больший список задать вопрос извинить зря стоять очередь ребёнок подсказать продажа распространить лекарство который обычно фармацевт помнить память отреагирова

Save on Google Drive

In [70]:
with open('/content/drive/MyDrive/NLP_FU/tokenized_inputs.pkl', 'wb') as f:
    pickle.dump(tokenized_inputs, f)

Read from Google Drive

In [71]:
with open('/content/drive/MyDrive/NLP_FU/tokenized_inputs.pkl', 'rb') as f:
    tokenized_inputs = pickle.load(f)

### Dataset

In [11]:
# dataset = TensorDataset(tokenized_inputs, torch.tensor(df_reviews['rating'].values, dtype = torch.long))
# train_size = len(dataset) * 0.8
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# # Создание DataLoader для эффективной обработки данных во время обучения
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [72]:
class MyDataset(Dataset):

    def __init__(self, all_reviews, labels):
        self.all_reviews = all_reviews
        self.labels = labels

    def __getitem__(self, ids):

        return {
            'input_ids' : self.all_reviews[ids]['input_ids'].squeeze().to(torch.long),
            'token_type_ids' : self.all_reviews[ids]['token_type_ids'].squeeze().to(torch.long),
            'attention_mask' : self.all_reviews[ids]['attention_mask'].squeeze().to(torch.long),
            'labels' : torch.tensor(self.labels.values[ids], dtype=torch.long)
        }

    def __len__(self):
        return len(self.all_reviews)

In [73]:
dataset = MyDataset(tokenized_inputs, df_reviews['rating'])
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [74]:
tokenized_inputs[1]['input_ids'].squeeze().to(torch.long).dtype

torch.int64

In [75]:
dataset[1]['labels'].dtype

torch.int64

### DataLoader

In [76]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

### DeepPavlov Model

In [27]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence", num_labels=5).to(device)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

### Train

In [28]:
n_epochs = 3

optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [77]:
loss_train = []
accuracy_train = []


for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_dataloader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):


        optimizer.zero_grad()
        input_ids, token_type_ids, attention_mask, labels =  batch['input_ids'], batch['token_type_ids'] , batch['attention_mask'], batch['labels']
        print("Tensor on Device:", input_ids.device, attention_mask.device, labels.device)
        print("Tensor on Shape:", input_ids.dtype, attention_mask.dtype, labels.dtype)
        print("Tensor on Shape:", input_ids.shape, attention_mask.shape, labels.shape)



        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        labels = labels.to(device)

        model.zero_grad()


        print("Tensor on Device:", input_ids.device, attention_mask.device, labels.device)
        print("Tensor on Shape:", input_ids.dtype, attention_mask.dtype, labels.dtype)

        print("Model on Device:", next(model.parameters()).device)
        loss, outputs = model(input_ids,
                             token_type_ids=None,
                             attention_mask=attention_mask,
                             labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            total += (predictions == labels).sum().item()
            correct += len(labels)

    accuracy = total / correct
    print(f"Accuracy on test set: {accuracy}")

Training epoch 1/3:   0%|          | 0/12071 [00:00<?, ?it/s]

Tensor on Device: cpu cpu cpu
Tensor on Shape: torch.int64 torch.int64 torch.int64
Tensor on Shape: torch.Size([4, 64]) torch.Size([4, 64]) torch.Size([4])


RuntimeError: ignored

In [78]:
print(torch.isnan(input_ids).any(), torch.isinf(input_ids).any())
print(torch.isnan(attention_mask).any(), torch.isinf(attention_mask).any())
print(torch.isnan(labels).any(), torch.isinf(labels).any())


tensor(False) tensor(False)
tensor(False) tensor(False)
tensor(False) tensor(False)


In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 66

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)
    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


## Задание 3. Примените один из трансформеров, например BERT, к задаче генерации англоязычного и русскоязычного текстов. Сравните результаты с LSTM. Сделайте выводы.

## Задание 4. Примените один из трансформеров, например BERT, к задаче машинного перевода.